# STEP 1-3

Below you see a list of so-called "libraries". Python does not know a lot by itself. By importing the libraries, Python is able to do all sorts of things such as making a plot.

In [ ]:
#Importing required libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import learning_curve
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import VarianceThreshold
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(12,6)})
import warnings
warnings.filterwarnings('ignore')


#importing dataset
wine = pd.read_csv("winequality-red_2.csv") 
wine_copy = pd.read_csv("winequality-red_2.csv") 

# STEP 4: Data exploration 

Look through the table `wine` and explore the variables. Remember that you can look at the dataset with the `.head()` function. If you use `.head(n=20)` you can see the first 20 variables. 

Look at the distribution of the dataset with `.describe()`. The function shows several summarizing values of the column, such as the mean of the column, the smallest value of the column (min) and the largest value (max). Pay attention to anything that stands out! 

Look through the table of `wine` and fill out below the variables below. 

In [ ]:
number_of_variables = 
mean_quality = 
minimum_alchol = 
maximum_pH = 

We want to predict the quality of wine, therefore `"quality"` is our target column. Before we are going to do any modelling, we first want to see the relationship between quality and the other columns. Are there any visible patterns? 

Let's explore the relationship between our target variable and the features. By the code below we plot quality with respect to alcohol with the help of `sns.barplot()`. `x` indicates the variable on the x-axis, `y` the y-axis. Change the `y` to other column names in order to explore the relationship with other features, e.g. `y = 'fixed acidity'` or `'citric acidity'`. Don't forget `''`!


In [ ]:
sns.barplot(x = 'quality', y = 'alcohol', data = wine)

# STEP 5: Feature engineering
## 5.1 Dealing with missing data

In the data exploration you could notice several missing values. Let's get an overview. The function `.isna()` stands for is not available. It returns all empty the empty cells. `.sum()` counts all these empty cells per column. 

In [ ]:
wine.isna().sum()

The missing values are present in the price column. We want to get rid of the missing values. There are several ways to approach this. However, replacing the missing values by zeros is often not a good idea. Below, we demonstrate why. First we plot the distribution of the price column, without removing the not available values. We do this by using the `.hist()` function

In [ ]:
wine["price"].hist()

 `.fillna()` stands for fill not availables. It replaces the not available values by any number that we put between the brackets. Now, we'll make the mistake of replacing missing values by zeros. 

In [ ]:
wine["price"].fillna(0).hist()

As you can see, our distribution changed drastically. This will disturb the data and can potentially lead to the wrong conclusions. Other ways to deal with missing data is simply dropping the rows with missing values. However this leads to loss of valuable information. This is definitely not a good idea when there are many missing values. You will decrease the size of your dataset drastically.

Another approach is replacing missing values by the mean of the column. We will do this for now. `.fillna()` stands for fill not availables. It replaces the not available values by any number that we put between the brackets. `.mean()` returns the column mean. 

In [ ]:
wine["price"] = wine["price"].fillna(wine["price"].mean())

wine["price"].hist()

#### Optional reading material: 
You can see that that the shape of the distribution stays relatively similar compared to the distribution with missing values. However, it should be noted that it could also lead to trouble, as it biases the data towards the mean. This mean might not apply to the real life situation. Therefore, it may lead to inaccurate model predictions. Other statistical approaches are available to deal with missing data, but we will keep it for now with the mean replacement! 

## 5.2 Making quality binary: good and bad quality

We are going to divide the data into two bins: one with good wines and one with bad wines. We will use the quality grade to divide the data. Wines graded below 5.5 will be placed in the bad quality bin, wines graded above 5.5 will be placed in the good quality bin. This boundary is arbitrary. If you want to set a higher standard, replace the value "5.5" of the variable `bins`. 

The `pd.cut()` function replaces the values above 5.5 with `"good"` and the values below 5.5 with `"bad"`. 

In [ ]:
#Dividing wine as good and bad by giving the limit for the quality
bins = (1, 5.5, 10)
group_names = ['bad', 'good']
wine['quality'] = pd.cut(wine['quality'], bins = bins, labels = group_names)

We now look through our quality columns to see how many of the wines in our data set our considered good versus bad!

In [ ]:
sns.countplot(wine['quality'])

Our modelling algorithm only understand numbers, no words. Therefore we need to replace the labels "good" and "bad" by numbers. We set bad to 0 and good to 1. Therefore we use an inbuilt functionality of one of our libraries called `LabelEncoder()`.  The `LabelEncoder()` automatically replaces bad with 0 and good with 1.

In [ ]:
label_quality = LabelEncoder()
 
wine['quality'] = label_quality.fit_transform(wine['quality'])

# STEP 6: Feature selection

## 6.1 Removing features by common sense 

Look again through the data table. Some features might be unnecessary. With common sense we can say that they do not influence the quality of wine. These colomns should be removed, as they provide no predictive information of quality for our model. Write down the column names below, with the following structure `["feature1", "feature2", "feature3"]`

In [ ]:
unrelated_features =

#this function ensures that the unrelated features are dropped 
wine = wine.drop(columns = unrelated_features, axis=1)

## 6.2 Removing collinear features

The plot `sns.heatmap()` below shows whether the variables of the columns of our dataset correlate, e.g. whether price corresponds with alcohol. The correlation varies between the 1 and -1. 0 means no correlation. 1 means perfect positive correlation and -1 perfect negative correlation. 

Lighter colours show a positive correlation, e.g. higher alcohol content might relate to a higher price. Dark colours show a negative correlation, e.g. higher alcohol content might relate to a lower price. Purple means no correlation, e.g. price and alcohol are unrelated. The values of the correlation are displayed by setting `annot` (annotation) to `True`. 

We use a copy of our dataset `wine_copy` that we made in the beginning. In our `wine` dataset we changed the quality values to 0 and 1. This affects the correlation. Therefore we look at the correlation before we made this change! 

In [ ]:
# calculate the correlation between each column
corr = wine_copy.corr()

#round the correlation on with 2 numbers behind the comma
corr = round(corr, 2)

# plot the heatmap
sns.heatmap(corr, xticklabels=wine_copy.columns, yticklabels=wine_copy.columns, annot=True, annot_kws = {"size" : 8 }, linewidths=.5)

Which column is best to remove? 

In [ ]:
correlated_features = 

wine = wine.drop(columns = correlated_features, axis=1)

## 6.3 Low variance filter 

Consider a variable in our dataset where all the observations have the same value, e.g. alcohol is always 15%. If we use this variable, do you think it can adds informations? The answer is no, because this variable has zero variance. A change in alcohol would not lead to a change in quality, because alcohol is a constant of 15%. To detect this, python can calculate the variance of each variable we are given. The function `.var()` shows the variance of each of the columns in the dataset.

In [ ]:
wine.var()

We set a threshold, a point where we distinguish between high and low variance. We want to set the threshold close to zero, which means that the variable stays almost the same. Higher thresholds can be set, however, you will loose a lot of data that might have valuable information about your target variable.

In [ ]:
threshold = 0.005

With the use of the `>`-statement you can check if the variance of the columns is higher that the threshold. 

In [ ]:
wine.var() > threshold

Plug this statement into the `.loc[]` function below. It will return our dataset with only those columns that have a variance higher than our threshold. In other words: it will return only the columns that are `True` following the `>` statement. 

In [ ]:
#return all columns with variance higher than the threshold
wine = wine.loc[:, "put the statement here"] 

# STEP 7: Modelling

## 7.1 Target and feature setting

Another preperation for the modelling is seperating your target and features and storing them to a `X` and `y` variable. Our target `y` is the binary value quality.The 0 represents bad quality and the 1 represents good quality wine. 

In [ ]:
y = wine['quality']

y.head()

Review the final features `X`.

In [ ]:
X = wine.drop(['quality'], axis = 1)

X.head()

## 7.2 Splitting test and train data

In order to assess your model’s performance later, you will need to divide the dataset into two parts: a training set and a test set.

The first is used to train the model, while the second is used to evaluate our trained model. In practice, the division of your data set into a test and a training sets is disjoint: the most common splitting choice is to take 70% of your original data set as the training set, while the 30% that remains will compose the test set.


The function `train_test_split()` divides our feature data (`X`) in two parts and our target data (`y`) in two parts. 
'test_size' is set to 0.3, which means that 30% of our data go to the test set. You can change `test_size` to a different number to see the effect on the results later. `random_state` ensures that it will always return the same split of test and train set. We set it to 42, but it can be any number.  


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

## 7.3 Algorithms

Finally, it is time for modelling! Usually the collection, exploration and preperation of the data, requires much more time then the modelling, as data is rarely perfect. We will be testing three different models: a decision tree, random forest and logistic regression. 


### Decision Tree

First we define our model. We use `tree.DecisionTreeClassifier()`. Next we are going to use our training data `X_train` and `y_train` to fit our model parameters with `.fit()`. The training of the model parameters is not visible! A lot goes on in the background! 

In [ ]:
#Defining the model
dtc = tree.DecisionTreeClassifier()

#fit the model to our training data
dtc.fit(X_train, y_train)

Next, we are making predictions of our test set, based on our fitted model. Remember `dtc` is the value where our decision tree model is specified. We use the `.predict()` function and apply this to make predictions about our test set `X_test`.

In [ ]:
#Making prediction on the test set 
pred_dtc = dtc.predict(X_test)

We will take the same steps for our 2 other models: random forest and logistic regression. 

### Random Forest

In [ ]:
#Defining the model
rfc = RandomForestClassifier(n_estimators=200)

#fit the model to our training data
rfc.fit(X_train, y_train)

In [ ]:
#Making prediction on the test set 
pred_rfc = rfc.predict(X_test)

### Logistic Regression

In [ ]:
#Defining the model
lrg = LogisticRegression()

#fit the model to our training data
lrg.fit(X_train, y_train)

In [ ]:
#Making prediction on the test set 
pred_lrg = lrg.predict(X_test)

# STEP 8: Evaluation


## 8.1 Confusion matrix

To evaluate the amount of mistakes made by the models, we are going to look at the confusion matrix. We use the `confusion_matrix()` function and create a table for the values. We add some names to the columns, to clarify the numbers. Look through the table and calculate the accuracy score. We do this by dividing the correct predictions by the total. 

### Decision tree

In [ ]:
#Displaying the confusion matrix
cm_dtc = pd.DataFrame(
    confusion_matrix(y_test, pred_dtc), 
    index=['true: bad quality', 'true: good quality'], 
    columns=['prediction: bad quality', 'prediction: good quality']
)

cm_dtc

In [ ]:
accuracy_dtc = 

### Random forest

In [ ]:
#Displaying the confusion matrix
cm_rfc = pd.DataFrame(
    confusion_matrix(y_test, pred_rfc), 
    index=['true: bad quality', 'true: good quality'], 
    columns=['prediction: bad quality', 'prediction: good quality']
)

cm_rfc

In [ ]:
accuracy_rfc = 

### Logistic regression

In [ ]:
#Displaying the confusion matrix
cm_lrg = pd.DataFrame(
    confusion_matrix(y_test, pred_lrg), 
    index=['true: bad quality', 'true: good quality'], 
    columns=['prediction: bad quality', 'prediction: good quality']
)

cm_lrg

In [ ]:
accuracy_lrg = 

## Storing our clean data
We want to store our clean data set, so we can use it for the next module! This will ensure that we do not have to do our work twice! 

In [ ]:
wine.to_csv("winequality-red_3.csv",index=False)